<a href="https://colab.research.google.com/github/ThanhHung2112/LMS/blob/main/Fact_checkipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets tqdm pandas
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from transformers import BertTokenizer, BertModel
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from datasets import DatasetDict

In [ ]:
# mnli = nlp.load_dataset(path='glue', name='mnli')
from datasets import load_dataset

# Tải dữ liệu từ MultiNLI
dataset = load_dataset("multi_nli")

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [ ]:
split_ratio = 1

total_train = len(dataset["train"])
total_val = len(dataset["validation_matched"])

# train_size = int(total_train * (1 - split_ratio))
validation_size = int(total_val  * split_ratio *0.5 )

# Truy cập các phần khác nhau của dữ liệu
# train_data = dataset["train"].shuffle(seed=42).select([i for i in range(train_size)])
# validation_data = dataset["validation_matched"].shuffle(seed=42).select([i for i in range(validation_size)])
# test_data = dataset["validation_matched"].shuffle(seed=42).select([i for i in range(total_val - 1, validation_size - 1, -1)])

train_data = dataset["train"].shuffle(seed=42)
validation_data = dataset["validation_matched"].shuffle(seed=42).select([i for i in range(validation_size)])
test_data = dataset["validation_matched"].shuffle(seed=42).select([i for i in range(total_val - 1, validation_size - 1, -1)])

# Tạo DatasetDict chứa các tập dữ liệu con
splits = DatasetDict({
    "train": train_data,
    "validation": validation_data,
    "test": test_data
})

print("Number of training examples:", len(splits["train"]))
print("Number of validation examples:", len(splits["validation"]))
# In số lượng ví dụ trong từng phần
print("Number of training examples:", len(splits["train"]))
print("Number of validation examples:", len(splits["validation"]))
print("Number of test examples:", len(splits["test"]))

# Truy cập các ví dụ cụ thể
sample_example = train_data[0]
print("Example premise:", sample_example["premise"])
print("Example hypothesis:", sample_example["hypothesis"])
print("Example label:", sample_example["label"]) # [0,1,2] -- [entailment, neutral, contradiction]



Number of training examples: 392702
Number of validation examples: 4907
Number of training examples: 392702
Number of validation examples: 4907
Number of test examples: 4908
Example premise: I'll hurry over that part.
Example hypothesis: "I'll be quick with that part."
Example label: 0


In [ ]:
# ------------------
# Sử dụng tokenizer của BERT để mã hoá dữ liệu
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_seq_length = 128

# Chuẩn bị dữ liệu dạng Tensor
premises = [sample['premise'] for sample in train_data]
hypotheses = [sample['hypothesis'] for sample in train_data]
labels = [sample['label'] for sample in train_data]

tokenized_premises = tokenizer(premises, padding=True, truncation=True, max_length=max_seq_length, return_tensors='pt')
tokenized_hypotheses = tokenizer(hypotheses, padding=True, truncation=True, max_length=max_seq_length, return_tensors='pt')

input_ids = torch.cat((tokenized_premises['input_ids'], tokenized_hypotheses['input_ids']), dim=1)
attention_mask = torch.cat((tokenized_premises['attention_mask'], tokenized_hypotheses['attention_mask']), dim=1)
labels = torch.tensor(labels)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_input_ids, test_input_ids, train_attention_mask, test_attention_mask, train_labels, test_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42)


In [ ]:
# Xây dựng mô hình
class BERTClassificationModel(nn.Module):
    def __init__(self, bert_model_name, num_labels):
        super(BERTClassificationModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size * 2, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = torch.cat((outputs.last_hidden_state[:, 0, :], outputs.last_hidden_state[:, -1, :]), dim=1)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [ ]:
# Huấn luyện mô hình
model = BERTClassificationModel('bert-base-uncased', num_labels=3)  # 3 classes: entailment, neutral, contradiction
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tạo DataLoader cho tập huấn luyện
batch_size = 32
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Hàm mất mát và tối ưu hóa
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Huấn luyện mô hình

In [ ]:
# Huấn luyện mô hình
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss}")

Epoch 1/1, Loss: 5577.226287513971


In [ ]:
# Đánh giá mô hình trên tập kiểm tra
model.eval()
test_dataset = TensorDataset(test_input_ids, test_attention_mask)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
all_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)

        logits = model(input_ids, attention_mask)
        probabilities = torch.softmax(logits, dim=1)
        predicted_labels = torch.argmax(probabilities, dim=1)
        all_predictions.extend(predicted_labels.cpu().numpy())

In [ ]:
print("Classification Report:")
print(classification_report(test_labels.numpy(), all_predictions))

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.84      0.84     26207
           1       0.80      0.76      0.78     26082
           2       0.81      0.85      0.83     26252

    accuracy                           0.82     78541
   macro avg       0.82      0.82      0.82     78541
weighted avg       0.82      0.82      0.82     78541



In [ ]:
bert_fact = model
bert_tokenizer = tokenizer

In [ ]:
bert_tokenizer.save_pretrained("/content/drive/MyDrive/bert_fact_check/tokenizer")

In [ ]:
# Lưu mô hình vào tệp
model_path = "/content/drive/MyDrive/bert_fact_check/bert_classification_model.pth"
torch.save(bert_fact.state_dict(), model_path)

In [ ]:
import torch
model_path = "/content/drive/MyDrive/bert_fact_check/bert_classification_model.pth"
# Tạo mô hình mớia
loaded_model = BERTClassificationModel('bert-base-uncased', num_labels=3)

# Nạp trạng thái từ tệp đã lưu
loaded_model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)

BERTClassificationModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [ ]:
from transformers import BertTokenizer

# Tải lại tokenizer từ đường dẫn đã lưu
loaded_tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/bert_fact_check/tokenizer")

In [ ]:
import torch
from transformers import BertTokenizer
max_seq_length = 128
def predict_premise_hypothesis(premise_text, hypothesis_text, model, tokenizer):
    # Chuẩn bị dữ liệu đầu vào cho mô hình
    inputs = tokenizer(premise_text, hypothesis_text, padding=True, truncation=True, max_length=max_seq_length, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Dự đoán
    model.eval()
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probabilities = torch.softmax(logits, dim=1)
        predicted_label = torch.argmax(probabilities, dim=1).item()

    return predicted_label

pre = ["entailment", "neutral", "contradiction"]
# Sử dụng hàm predict_premise_hypothesis để dự đoán
premise_text = "The likelihood is 100%"
hypothesis_text = "Maybe the probability is 100%"
predicted_label = predict_premise_hypothesis(premise_text, hypothesis_text, loaded_model, loaded_tokenizer)

print("premise_text :", premise_text, "\n", "hypothesis_text :", hypothesis_text)
print("Predicted Label:", pre[predicted_label])


premise_text : The likelihood is 100% 
 hypothesis_text : Maybe the probability is 100%
Predicted Label: entailment


In [ ]:
premise_text = "The dataset had acquired for the student of 7 computer science"
hypothesis_text = """
the dataset has been acquired from the official IIIT-Delhi academics department for the students of 7 Computer Science passout batches.
the dataset consists 739 students and 306 subjects with mapping of each student to the grades for each course the student has taken throughout the duration of their degree.
"""
predicted_label = predict_premise_hypothesis(premise_text, hypothesis_text, loaded_model, loaded_tokenizer)
print("premise_text :", premise_text, "\n", "hypothesis_text :", hypothesis_text)
print("Predicted Label:", pre[predicted_label])

premise_text : Tôi thích cô ấy 
 hypothesis_text : Tôi yêu cô ấy
Predicted Label: contradiction
